In [1]:
import mxnet as mx
import os
import random
from mxnet.image import ImageIter

In [2]:
data_dir = "./data/Caltech256/"

In [3]:
!curl -O https://raw.githubusercontent.com/dmlc/mxnet/master/tools/im2rec.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13352  100 13352    0     0  47233      0 --:--:-- --:--:-- --:--:-- 53195


In [4]:
%run -i 'im2rec.py --list=true'

ERROR:root:File `'im2rec.py --list=true.py'` not found.


In [3]:
def list_image(root, recursive, exts):
    image_list = []
    i = 0
    if recursive:
        cat = {}
        for path, subdirs, files in os.walk(root):
            for fname in files:
                fpath = os.path.join(path, fname)
                suffix = os.path.splitext(fname)[1].lower()
                if os.path.isfile(fpath) and (suffix in exts):
                    if path not in cat:
                        label = path.split("/")[-1]
                        cat[path] = int(label) #use the last digit as the class ID#print cat[path]
                    image_list.append((i, os.path.relpath(fpath, root), cat[path]))
                    i += 1
    else:
        for fname in os.listdir(root):
            fpath = os.path.join(root, fname)
            suffix = os.path.splitext(fname)[1].lower()
            if os.path.isfile(fpath) and (suffix in exts):
                image_list.append((i, os.path.relpath(fpath, root), 0))
                i += 1
    return image_list
def make_list(image_list, shuffle=True, chunks=1, prefix="pre", train_ratio=0.65, test_ratio=0.2):
#     image_list = list_image(args.root, args.recursive, args.exts)
#     image_list = list(image_list)
    if shuffle is True:
        random.seed(100)
        random.shuffle(image_list)
    N = len(image_list)
    chunk_size = (N + chunks - 1) // chunks
    for i in range(chunks):
        chunk = image_list[i * chunk_size:(i + 1) * chunk_size]
        if chunks > 1:
            str_chunk = '_%d' % i
        else:
            str_chunk = ''
        sep = int(chunk_size * train_ratio)
        sep_test = int(chunk_size * test_ratio)
        if train_ratio == 1.0:
            write_list(prefix + str_chunk + '.lst', chunk)
        else:
            if test_ratio:
                write_list(prefix + str_chunk + '_test.lst', chunk[:sep_test])
            if train_ratio + test_ratio < 1.0:
                write_list(prefix + str_chunk + '_val.lst', chunk[sep_test + sep:])
            write_list(prefix + str_chunk + '_train.lst', chunk[sep_test:sep_test + sep])
def write_list(path_out, image_list):
    with open(path_out, 'w') as fout:
        print(path_out)
        for i, item in enumerate(image_list):
            line = '%d\t' % item[0]
            for j in item[2:]:
                line += '%f\t' % j
            line += '%s\n' % item[1]
            fout.write(line)

In [4]:
!pwd

/home/ec2-user/learn-dl/convolution


In [5]:
img_list = list_image("data/Caltech256/", True, ['.jpg', '.jpeg'])

In [6]:
!pwd

/home/ec2-user/learn-dl/convolution


In [10]:
%cd learn-dl/convolution/data/

/home/ec2-user/learn-dl/convolution/data


In [11]:
make_list(img_list)

pre_test.lst
pre_val.lst
pre_train.lst


In [18]:
img_iter = ImageIter(batch_size=4, data_shape=(3, 224, 224), label_width=1, path_imglist="pre_test.lst", path_root="data/Caltech256/")

loading image list...


In [19]:
import os, urllib
def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.urlretrieve(url, filename)

In [21]:
!pwd

/home/ec2-user/learn-dl/convolution


In [22]:
!wget "http://data.mxnet.io/models/imagenet/vgg/vgg16-0000.params"
!wget "http://data.mxnet.io/models/imagenet/vgg/vgg16-symbol.json"

--2017-07-03 18:42:08--  http://data.mxnet.io/models/imagenet/vgg/vgg16-0000.params
Resolving data.mxnet.io (data.mxnet.io)... 54.208.175.7
Connecting to data.mxnet.io (data.mxnet.io)|54.208.175.7|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 553431792 (528M)
Saving to: ‘vgg16-0000.params’

vgg16-0000.params   100%[===================>] 527.79M  11.3MB/s    in 42s     

2017-07-03 18:42:50 (12.5 MB/s) - ‘vgg16-0000.params’ saved [553431792/553431792]

--2017-07-03 18:42:50--  http://data.mxnet.io/models/imagenet/vgg/vgg16-symbol.json
Resolving data.mxnet.io (data.mxnet.io)... 54.208.175.7
Connecting to data.mxnet.io (data.mxnet.io)|54.208.175.7|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16554 (16K) [application/json]
Saving to: ‘vgg16-symbol.json’

vgg16-symbol.json   100%[===================>]  16.17K  --.-KB/s    in 0.08s   

2017-07-03 18:42:51 (198 KB/s) - ‘vgg16-symbol.json’ saved [16554/16554]



In [26]:
sym, arg_params, aux_params = mx.model.load_checkpoint('vgg16', 0)

In [40]:
all_layers = sym.get_internals()

In [69]:
tmp = all_layers['prob_label']

In [70]:
tmp.get_children()

In [58]:
net = all_layers['fc8']
net = mx.symbol.FullyConnected(data=net, num_hidden=257, name='out')
net = mx.symbol.SoftmaxOutput(data=net, name='softmax')
new_args = dict({k: arg_params[k] for k in arg_params if 'out' not in k})

ValueError: Cannot find output that matches name "fc8"

In [44]:
train_iter = ImageIter(batch_size=32, data_shape=(3, 224, 224), label_width=1, path_imglist="pre_train.lst", path_root="data/Caltech256/")
val_iter = ImageIter(batch_size=32, data_shape=(3, 224, 224), label_width=1, path_imglist="pre_val.lst", path_root="data/Caltech256/")

loading image list...
loading image list...


In [54]:
net.get_internals()

<Symbol group [fc8_weight, out_weight, out_bias, out, softmax_label, softmax]>

In [50]:
mx.mod.Module?

In [49]:
mod = mx.mod.Module(symbol=net, context=[mx.gpu(0)])

ValueError: [91mYou created Module with Module(..., data_names=['data']) but input with name 'data' is not found in symbol.list_arguments(). Did you mean one of:
	softmax_label[0m

In [48]:
net

<Symbol softmax>